# [Day 10] 속도의 미학: FPS 계산과 시스템 통합

3주차의 마지막 날입니다.
우리는 카메라로 손을 인식하는 기술을 배웠습니다. 하지만 로봇을 제어할 때 가장 중요한 것은 **속도**입니다.

내가 손을 움직였는데 1초 뒤에 로봇이 움직인다면 답답해서 쓸 수 없겠죠?
오늘은 내 컴퓨터가 얼마나 빠르게 화면을 처리하고 있는지 측정하는 **FPS(초당 프레임 수)**에 대해 배우고, 앞으로 만들 시스템의 전체 구조를 잡아보겠습니다.

## 1. FPS (Frames Per Second)란?

게임할 때 많이 들어보셨죠? 1초에 화면을 몇 번이나 새로 그리는지를 나타내는 수치입니다.
- **30 FPS**: TV 방송 수준. 부드러워 보입니다.
- **60 FPS**: 아주 부드러운 게임 화면.
- **10 FPS 이하**: 뚝뚝 끊겨 보입니다. 로봇 제어에는 좋지 않습니다.

In [ ]:
import cv2 as cv
import time

# 간단한 FPS 계산기 클래스
class SimpleFPS:
    def __init__(self):
        self.prev_time = 0
    
    def update(self):
        new_time = time.time()
        # 현재 시간과 이전 시간의 차이(초 단위)를 구합니다.
        diff = new_time - self.prev_time
        self.prev_time = new_time
        
        # 시간 차이가 0이면(너무 빠르면) 계산 불가
        if diff == 0: return 0
        
        # 1초 나누기 시간차 = FPS
        fps = 1 / diff
        return int(fps)

## 2. 내 컴퓨터의 처리 속도 측정해보기

웹캠을 켜고 손 인식 모델을 돌리면서, 화면 왼쪽 위에 FPS를 띄워봅시다.

In [ ]:
import mediapipe as mp

cap = cv.VideoCapture(0)
fps_calc = SimpleFPS()

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2)
mp_drawing = mp.solutions.drawing_utils

while True:
    ret, frame = cap.read()
    if not ret: break
    
    frame = cv.flip(frame, 1)
    image_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    
    # --- 여기서 인공지능이 계산을 수행합니다 ---
    results = hands.process(image_rgb)
    # ----------------------------------------
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # FPS 계산
    fps = fps_calc.update()
    
    # 화면에 FPS 표시 (초록색 글씨)
    cv.putText(frame, f"FPS: {fps}", (10, 30), 
               cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv.imshow('FPS Check', frame)
    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

## 3. 로봇 제어 파이프라인 (Pipeline)

우리가 앞으로 만들 프로그램의 전체 흐름은 다음과 같습니다. 마치 공장 라인처럼 데이터가 흘러갑니다.

1.  **입력 (Input)**: 웹캠이 사진을 찍습니다. (Day 6)
2.  **인식 (Recognition)**: MediaPipe가 손의 위치(좌표)를 찾습니다. (Day 8~9)
3.  **계산 (Calculation)**: 손가락이 얼마나 굽혀졌는지 **각도**를 계산합니다. (다음 주 예고!)
4.  **통신 (Communication)**: 계산된 각도를 로봇에게 전송합니다. (5주차)
5.  **동작 (Actuation)**: 로봇 손이 실제로 움직입니다.

이 모든 과정이 0.03초(30 FPS) 안에 일어나야 로봇이 내 손처럼 자연스럽게 움직입니다.

## 4. 3주차 마무리 및 예고

이제 로봇의 '눈'과 '지능'을 완성했습니다.
다음 주(4주차)부터는 **수학(Math)** 시간이 조금 기다리고 있습니다.

"수학이라니!" 하고 겁먹지 마세요.
점 3개만 있으면 각도를 구할 수 있다는 아주 간단한 원리부터 차근차근 배워볼 것입니다.
이 각도를 구해야 비로소 로봇 모터를 제어할 수 있거든요.

주말 동안 푹 쉬시고, 활기찬 모습으로 다음 주에 뵙겠습니다!